# Performance e Operações Paralelas

In [3]:
using Pkg

using BenchmarkTools
using LoopVectorization
using Plots
using StaticArrays
using ThreadsX

using CUDA # há vá?!?!

using Random: seed!, shuffle
using Statistics: mean

# Random Seed
seed!(123)

# Printar cores no terminal
using ANSIColoredPrinters

# Checklist basico para _performance_

1. Arrumar instabilidade de tipo
2. Usar variáveis locais ao invés de gloabais
3. Deixar tudo imutável se possível
4. Desativar checagem de índice em operações com `Array`
5. Ativar suporte SIMD em todos os loops `for` (Single Instruction Multiple Data)

# 1. Arrumar instabilidade de tipo

> Tipo de saída de uma função é __imprevisível__ a partir dos tipos de entradas. Em particular, isso significa que o tipo de saída __pode variar__ dependendo dos valores das entradas.

In [4]:
function positivo(x)
    if x > 0
        return x
    else
        return 0
    end
end

positivo (generic function with 1 method)

> função com untabilidade de tipo >>> `x` é o que?

`@code_warntype` avalia a função como um argumento e prenta um _Abstract Syntax Tree_ (AST)

In [5]:
@code_warntype positivo(-3.4)

Variables
  #self#::Core.Const(positivo)
  x::Float64

Body::Union{Float64, Int64}
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0


> onde ficou vermelho, tem algum problema de tipo

### Arrumando a instabilidade de tipo

Anotar os tipos!

In [6]:
function positivo_stabel(x::AbstractFloat)
    if x > 0
        return x 
    else
        return 0.0
    end
end

positivo_stabel (generic function with 1 method)

In [7]:
function positivo_stabel(x::Integer)
    if x > 0
        return x 
    else
        return 0
    end
end

positivo_stabel (generic function with 2 methods)

In [8]:
@code_warntype positivo_stabel(-3.4)

Variables
  #self#::Core.Const(positivo_stabel)
  x::Float64

Body::Float64
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0.0


In [9]:
@code_warntype positivo_stabel(-3)

Variables
  #self#::Core.Const(positivo_stabel)
  x::Int64

Body::Int64
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0


### Pq é importante anotar os tipos?

In [11]:
x = rand(1_000);

In [13]:
@benchmark positivo.($x)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.930 μs …  1.357 ms  ┊ GC (min … max):  0.00% … 99.49%
 Time  (median):     3.980 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   7.814 μs ± 37.978 μs  ┊ GC (mean ± σ):  17.16% ±  3.58%

  ▄▁ ▆█▅▃▃▃▃▂     ▄▃▄▅▃▂▁                                    ▁
  █████████████▇▇██████████████▇▇▇▇█▇▆▆▄▆▅▆▆▅▄▅▄▄▅▄▄▄▄▂▃▃▃▃▅ █
  1.93 μs      Histogram: log(frequency) by time     26.5 μs <

 Memory estimate: 8.22 KiB, allocs estimate: 7.

In [14]:
@benchmark positivo_stabel.($x)

BenchmarkTools.Trial: 10000 samples with 186 evaluations.
 Range (min … max):  803.763 ns … 348.528 μs  ┊ GC (min … max):  0.00% … 96.99%
 Time  (median):       1.267 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.478 μs ±   6.365 μs  ┊ GC (mean ± σ):  20.15% ± 13.30%

  ▇█▆▄▃▃▂▁▂▂▁▁                                                  ▁
  ███████████████▇▇▆▅▆▆▇▆▅▅▅▅▅▄▆▆▆▆▅▅▆▆▆▆▅▅▅▄▅▅▄▅▅▅▅▄▅▅▅▃▃▃▃▄▃▃ █
  804 ns        Histogram: log(frequency) by time       21.8 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

### Dicas!

Dar preferência para usar __tipos abstrtatos__ ao invés de tipos concretos.

`AbstractFlosa` >>> `Float64` or `Float32`  
`Integer` >>> `Int128` or `Int64` or `Int342`

### Tipos paramétricos

São introduzidos com as chaves `{}` e usando palavras-chave `where`

In [16]:
# por exemplo qualquer subtipo de `Real`
subtypes(Real)

8-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 FixedPointNumbers.FixedPoint
 Integer
 Rational
 StatsBase.PValue
 StatsBase.TestStat
 VectorizationBase.AbstractSIMD

In [17]:
function positivo_stable2(x::T) where T <: Real
    if x > 0
        return x
    else
        return 0::T
    end
end

positivo_stable2 (generic function with 1 method)

In [18]:
@code_warntype positivo_stable2(-3.4)

Variables
  #self#::Core.Const(positivo_stable2)
  x::Float64

Body::Float64
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      Core.typeassert(0, $(Expr(:static_parameter, 1)))
└──      Core.Const(:(return %4))


In [19]:
@code_warntype positivo_stable2(-3)

Variables
  #self#::Core.Const(positivo_stable2)
  x::Int64

Body::Int64
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─ %4 = Core.typeassert(0, $(Expr(:static_parameter, 1)))::Core.Const(0)
└──      return %4


### Funciona tambem com `Array`s

* `AbstractArray{T, N}` 
* `AbstractMatrix{T}` atalho para `AbstractArray{T, 2}` 
* `AbstractVector{T}` atalho para `AbstractArray{T, 1}` 

In [21]:
# específico para vector
function meus_zeros(X::AbstractVector{T}) where T <: Real
    return zeros(eltype(x), size(x))    
end

meus_zeros (generic function with 1 method)

In [22]:
# Generalizado para qualquer dimenmção de Array
function meus_zeros(X::AbstractArray{T, N}) where T <: Real where N <: Integer
    return zeros(eltype(x), size(x))    
end

meus_zeros (generic function with 2 methods)

In [25]:
@code_warntype meus_zeros([1, 0, 3])

Variables
  #self#::Core.Const(meus_zeros)
  X::Vector{Int64}

Body::Any
1 ─ %1 = Main.eltype(Main.x)::Any
│   %2 = Main.size(Main.x)::Any
│   %3 = Main.zeros(%1, %2)::Any
└──      return %3


# 2. Usar variáveis locais ao invés de gloabais

Com variáveis globais o compilador LLVM tem dificuldades em otimizar o código Assembly

In [30]:
# var global 
x = rand(1_000);

In [31]:
function sum_global()
    s = 0.0
    for i ∈ x 
        s += i
    end
    return s
end

sum_global (generic function with 1 method)

In [32]:
@benchmark sum_global()

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   78.300 μs …   5.809 ms  ┊ GC (min … max): 0.00% … 97.68%
 Time  (median):      80.200 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   109.528 μs ± 164.077 μs  ┊ GC (mean ± σ):  4.20% ±  3.04%

  █▄▂▁▂▁▁▂▁     ▃▁                                              ▁
  ██████████▇▇▇▇██▇▇██▆▇▇▆▇▇█▇▆▆▇▆▇▆▆▆▇▇▇▆▆▆▆▆▅▆▅▅▅▅▄▄▃▄▅▅▅▄▅▃▄ █
  78.3 μs       Histogram: log(frequency) by time        361 μs <

 Memory estimate: 70.16 KiB, allocs estimate: 3490.

In [33]:
function sum_arg(x)
    s = 0.0
    for i ∈ x 
        s += i
    end
    return s
end

sum_arg (generic function with 1 method)

In [34]:
@benchmark sum_arg($x)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.100 μs …  27.100 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.110 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.202 μs ± 513.365 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅▂▁ ▃▃▃▁▁                                                  ▁
  ███████████▇▇█▇▇▇▅██▅█▇▅▅▄▆▆▅▆▄▄▄▃▄▁▁▃▃▄▄▄▃▅▃▄▁▁▃▅▄▁▃▁▄▅▅▅▆ █
  1.1 μs       Histogram: log(frequency) by time      2.76 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [35]:
@code_warntype sum_global()

Variables
  #self#::Core.Const(sum_global)
  @_2::Any
  s::Any
  i::Any

Body::Any
1 ─       (s = 0.0)
│   %2  = Main.x::Any
│         (@_2 = Base.iterate(%2))
│   %4  = (@_2 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_2::Any
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Any
│         (s = s + i)
│         (@_2 = Base.iterate(%2, %9))
│   %12 = (@_2 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return s


In [36]:
@code_warntype sum_arg(x)

Variables
  #self#::Core.Const(sum_arg)
  x::Vector{Float64}
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  i::Float64

Body::Float64
1 ─       (s = 0.0)
│   %2  = x::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}::Tuple{Float64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (s = s + i)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return s


## Se tiver de usar variáveis globeis, use `const`

In [38]:
# var global constante
const const_x = rand(1_000);

In [39]:
function sum_const_global()
    s = 0.0
    for i ∈ const_x 
        s += i
    end
    return s
end

sum_const_global (generic function with 1 method)

In [40]:
@benchmark sum_const_global()

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.100 μs …  15.850 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.110 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.177 μs ± 422.373 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄▂▁ ▃▂▃▁                                                   ▁
  ██████████▆▆▆▇▇▆▄▃▃▄▅▃▃▃▄▃▃▁▃▁▄▃▁▁▃▁▃▁▃▁▃▃▁▁▄▄▃▃▁▃▁▄▃▃▅▅▅▆▆ █
  1.1 μs       Histogram: log(frequency) by time      2.76 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [41]:
@code_warntype sum_const_global()

Variables
  #self#::Core.Const(sum_const_global)
  @_2::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  i::Float64

Body::Float64
1 ─       (s = 0.0)
│   %2  = Main.const_x::Core.Const([0.16177249940502847, 0.06470205727455447, 0.22871236376036652, 0.556913457304788, 0.578843623826564, 0.5243372527452201, 0.5123207296427339, 0.8269323285749814, 0.8808212556704482, 0.7988402955861709, 0.5955474189908894, 0.3594057958548704, 0.42909377516866853, 0.2510995943330936, 0.339641850887098, 0.12419283006409554, 0.7716956508831458, 0.204475387594375, 0.8576726775715193, 0.8875257825543101, 0.11095576791526507, 0.9050631317153395, 0.756857868257089, 0.5572430416565493, 0.3422838517212543, 0.4122504738010766, 0.24031751887689712, 0.9628092785912741, 0.6122769927207683, 0.05213547983352762, 0.5832862892557842, 0.3089583085057257, 0.5061622722653056, 0.6571088693831761, 0.3305949639598311, 0.09541141551684529, 0.5749322205555325, 0.41320047819784045, 0.28057866886129257, 0.10079718819188477, 0.

# 3. Deixar tudo imutável se possível

3. Deixar tudo imutável se possível
4. Desativar checagem de índice em operações com `Array`
5. Ativar suporte SIMD em todos os loops `for` (Single Instruction Multiple Data)